In [1]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import os, sys # For filepaths
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.feature_selection import VarianceThreshold
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import label_binarize
import pickle


In [2]:
''' Add the datasets and libraries to the system path '''

# Find the path to our implementations
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
home_directory = os.path.dirname(parent_directory)
libraries_path = os.path.join(home_directory, 'Libraries')

# Find the path to the datasets
datasets_path = os.path.join(home_directory, 'Datasets')

# Add them both to the system path
sys.path.append(datasets_path)
sys.path.append(libraries_path)

In [3]:
''' Load the dataset '''

dataset = np.load(os.path.join(datasets_path, 'fashion_train.npy'))
print("Dataset shape: ", dataset.shape)

Dataset shape:  (10000, 785)


In [4]:
X = dataset[:, :-1]  # Define the images as all columns except the last one
y = dataset[:, -1] # Define the labels as the last column

In [5]:
print(type(X))
print("X shape: ", X.shape)
print(type(y))
print("y shape: ", y.shape)

<class 'numpy.ndarray'>
X shape:  (10000, 784)
<class 'numpy.ndarray'>
y shape:  (10000,)


## Code from Ania's FYP

In [6]:
# import classifiers 
from sklearn import svm 
from CNN import CNN
#from Bayes import Bayes_classifier
from sklearn.neighbors import KNeighborsClassifier as KNN

In [8]:
# Define classifiers
classifiers = [CNN(), KNN(5)]
classifier_names = ["CNN", "KNN"]
num_classifiers = len(classifiers)
num_folds = 5

group_kfold = KFold(n_splits=num_folds)

# Define functions to avoid code repetition
def get_metrics(classifiers, x, y):
    acc_val = np.empty([num_folds, num_classifiers])
    f1_val = np.empty([num_folds, num_classifiers])
    precision = np.empty([num_folds, num_classifiers])
    recall = np.empty([num_folds, num_classifiers])
    # roc_auc = np.empty([num_folds, num_classifiers])
    for i, (train_index, val_index) in enumerate(group_kfold.split(x, y)):
        x_train, x_val = x[train_index, :], x[val_index, :]
        y_train, y_val = y[train_index], y[val_index]
        for j, clf in enumerate(classifiers):
            print(f"Classifier {j+1} - {classifier_names[j]}")
            clf.fit(x_train, y_train)
            predictions = clf.predict(x_val)
            if classifier_names[j] == "CNN":
                predictions = np.array(predictions)
                predictions = predictions.astype(int)
                predictions = predictions.reshape(predictions.shape[0],)
            acc_val[i, j] = accuracy_score(y_val, predictions)
            f1_val[i, j] = f1_score(y_val, predictions, average='micro')
            precision[i, j] = precision_score(y_val, predictions, zero_division=0, average='micro')
            recall[i, j] = recall_score(y_val, predictions, average='micro')
    print_metrics(acc_val, f1_val, precision, recall)
    return x_train, y_train, x_val, y_val

def print_metrics(acc_val, f1_val, precision, recall, roc_auc):
    average_acc = np.mean(acc_val, axis=0)
    average_f1 = np.mean(f1_val, axis=0)
    average_precision = np.mean(precision, axis=0)
    average_recall = np.mean(recall, axis=0)
    for i, classifier_name in enumerate(classifier_names):
        print(f"############ Classifier {i+1} - {classifier_name}:")
        print(f'Average F1 score = {average_f1[i]:.3f}')
        print(f'Average Accuracy = {average_acc[i]:.3f}')
        print(f'Average Precision = {average_precision[i]:.3f}')
        print(f'Average Recall = {average_recall[i]:.3f}')

x_train, y_train, x_val, y_val = get_metrics(classifiers, X,y)

Classifier 1 - CNN
Classifier 2 - KNN
Classifier 1 - CNN


In [ ]:
# Normal feature selection
print("Running without feature selection")
x_train, y_train, x_val, y_val = get_metrics(classifiers, x, y, patient_id)

# Feature selection with variance threshold
print("Running with variance threshold feature selection")
threshold_value = 0.1
selector = VarianceThreshold(threshold=threshold_value)
x_selected = selector.fit_transform(x)
get_metrics(classifiers, x_selected, y, patient_id, "With variance threshold")

# Feature selection with PCA
print("Running with PCA feature selection")
pca_transformer = PCA(n_components=5)
x_pca = pca_transformer.fit_transform(x)
get_metrics(classifiers, x_pca, y, patient_id, "With PCA")

# Define the path where to save the file
pickle_path = 'Project-2_github_repo/Fixed/PROJECT/fyp2023/model_group02'

# Chosen classifier
classifier = KNN(n_neighbors=5)
classifier = classifier.fit(x, y)
filename = 'group02_classifier.sav'

# Check if the path exists, if not, create it
if not os.path.exists(pickle_path):
    os.makedirs(pickle_path)

pickle.dump(classifier, open(os.path.join(pickle_path, filename), 'wb'))


# Plot ROC curve for best classifier
clf = classifier
clf.fit(x_train, y_train)
y_score = clf.predict_proba(x_val)
fpr, tpr, _ = roc_curve(y_val, y_score[:, 1])
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title("Receiver Operating Characteristic")
plt.legend(loc="lower right")
plt.show()